# Clustering de Usuarios con K-Means (Interfaz Interactiva con ipywidgets)
Este notebook implementa un sistema de agrupamiento usando K-Means sobre un dataset de comportamiento de usuarios en redes sociales. Puedes ajustar el valor de K dinámicamente y visualizar los resultados.

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, IntSlider
import warnings
warnings.filterwarnings("ignore")

sns.set(style="whitegrid")


In [ ]:
# Cargar dataset
df = pd.read_csv("clustered_users.csv")

# Variables a usar para clustering
features = [
    'Daily_Usage_Time (minutes)', 'Posts_Per_Day', 'Likes_Received_Per_Day',
    'Comments_Received_Per_Day', 'Messages_Sent_Per_Day'
]
X = df[features]

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
@interact(k=IntSlider(min=2, max=10, step=1, value=4, description="Número de clusters (K):"))
def cluster_interactivo(k):
    # Aplicar K-Means
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_scaled)

    # Silhouette score
    sil_score = silhouette_score(X_scaled, clusters)

    # PCA para visualización
    pca = PCA(n_components=2)
    components = pca.fit_transform(X_scaled)
    df['Cluster'] = clusters
    df['PCA1'] = components[:, 0]
    df['PCA2'] = components[:, 1]

    # Gráfico
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=df, x='PCA1', y='PCA2', hue='Cluster', palette='Set2')
    plt.title(f'Clusters de Usuarios (K={k}) - Silhouette Score: {sil_score:.2f}')
    plt.xlabel("PCA 1")
    plt.ylabel("PCA 2")
    plt.show()

    # Estadísticas por cluster
    display(df.groupby('Cluster')[features].mean().style.background_gradient(cmap='Blues'))
